# OGGM workflow

# 1. Get rgi entities:
    Get a list of glacier putlines selected from their RGI IDs
## 2. Initialize glacier regions:
    Initializes a list of glacier directories for the run
## 3. GIS prepro tasks [tasks link](https://docs.oggm.org/en/latest/api.html#entity-tasks)
    Runs all flowline pre processing tasks including:
#### Glacier masks: Makes a gridded mask of the glacier outlines and topography.
This function fills holes in the source DEM and produces smoothed gridded topography and glacier outline arrays. These are the ones which will later be used to determine bed and surface height.
#### Compute centerlines: Compute the centerlines following Kienholz et al., (2014).
They are then sorted according to the modified Strahler number. Calculates routes along the topography and makes a shapely Linestring object from them.
#### Initialize flowlines: Computes more physical Inversion Flowlines from geometrical Centerlines
This interpolates the centerlines on a regular spacing (i.e. not the grid’s (i, j) indices. Cuts out the tail of the tributaries to make more realistic junctions. Also checks for low and negative slopes and corrects them by interpolation.
#### Compute downstream bedshape: The bedshape obtained by fitting a parabola to the line’s normals
Also computes the downstream’s altitude.
#### Catchment Area: Compute the catchment areas of each tributary line.
The idea is to compute the route of lowest cost for any point on the glacier to rejoin a centerline. These routes are then put together if they belong to the same centerline, thus creating “catchment areas” for each centerline.
#### Catchment_intersections: Computes the intersections between the catchments.
A glacier usually consists of several flowlines and each flowline has a distinct catchment area. This function calculates the intersections between these areas.
#### Catchment width geometry: Computes geometrical catchment widths for each point of the flowlines.
#### Catchment width correction: Corrects for NaNs and inconsistencies in the geometrical widths.
Interpolates missing values, ensures consistency of the surface-area distribution AND with the geometrical area of the glacier polygon, avoiding errors due to gridded representation.
## 4. Climate tasks
#### Process CRU data: Processes and writes the CRU baseline climate data for this glacier.
Interpolates the CRU TS data to the high-resolution CL2 climatologies (provided with OGGM) and writes everything to a NetCDF file.
#### Local t*: Compute the local t* and associated glacier-wide mu*.
Note: the glacier wide mu* is here just for indication. It might be different from the flowlines’ mu* in some cases.
#### Mu* calibration: Compute the flowlines’ mu* and the associated apparent mass-balance.
If low lying tributaries have a non-physically consistent Mass-balance this function will either filter them out or calibrate each flowline with a specific mu*. The latter is default and recommended.
## 5. Inversion tasks
#### Prepare for inversion: Prepares the data needed for the inversion
Mostly the mass flux and slope angle, the rest (width, height) was already computed. It is then stored in a list of dicts in order to be faster.
- #Flux needs to be in [m3 s-1] (*ice* velocity * surface)
- #fl.flux is given in kg m-2 yr-1, rho in kg m-3, so this should be it:
- rho = cfg.PARAMS['ice_density']
- flux = fl.flux * (gdir.grid.dx**2) / cfg.SEC_IN_YEAR / rho
#### Mass conservation inversion: Compute the glacier thickness along the flowlines
More or less following Farinotti et al., (2009).
#### Filter inversion output: Filters the last few grid point whilst conserving total volume.
The last few grid points sometimes are noisy or can have a negative slope. This function filters them while conserving the total volume.
